# Outlier Detection Benchmarks (OptiSLang-like) — Python Notebook

This notebook provides a reusable API to:
- Generate **Designs of Experiments (DoE)** for classic benchmarks (Branin, Rosenbrock, Hartmann, Kursawe)
- **Inject outliers** in input space and/or response space
- Run multiple **outlier detection** methods (IQR, Isolation Forest, LOF, One-Class SVM, Robust Covariance)
- Fit a **Gaussian Process** surrogate and use its **predictive variance** to estimate a **pre-simulation outlier risk**
- Visualize: scatter (2D), residual vs predicted (±σ bounds), IsolationForest score histogram, and GP variance contour (2D)

> You can switch benchmarks by changing `BENCHMARK_NAME` in the demo cell.

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Callable, Dict, Tuple

from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel as C
from sklearn.metrics import r2_score
from numpy.random import default_rng

# For Mahalanobis distance p-values (chi-square), fallback if scipy unavailable
try:
    from scipy.stats import chi2
    SCIPY_AVAILABLE = True
except Exception:
    SCIPY_AVAILABLE = False

rng = default_rng(42)


In [ ]:

from dataclasses import dataclass
from typing import Callable, Dict

import numpy as np

@dataclass
class Benchmark:
    name: str
    dim: int
    bounds: np.ndarray  # shape (dim, 2)
    f: Callable[[np.ndarray], np.ndarray]  # expects (n, dim) -> (n,) or (n,k)

def rosenbrock(x: np.ndarray, a: float = 1.0, b: float = 100.0) -> np.ndarray:
    x = np.atleast_2d(x)
    return np.sum(b * (x[:,1:] - x[:,:-1]**2)**2 + (a - x[:,:-1])**2, axis=1)

def branin(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    x1, x2 = x[:,0], x[:,1]
    a, b, c = 1.0, 5.1/(4*np.pi**2), 5/np.pi
    r, s, t = 6.0, 10.0, 1/(8*np.pi)
    return (a*(x2 - b*x1**2 + c*x1 - r)**2 + s*(1 - t)*np.cos(x1) + s)

def hartmann3(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    alpha = np.array([1.0, 1.2, 3.0, 3.2])
    A = np.array([[3.0, 10, 30],
                  [0.1, 10, 35],
                  [3.0, 10, 30],
                  [0.1, 10, 35]])
    P = 1e-4*np.array([[3689, 1170, 2673],
                       [4699, 4387, 7470],
                       [1091, 8732, 5547],
                       [381,  5743, 8828]])
    y = np.zeros(x.shape[0])
    for i in range(4):
        y -= alpha[i]*np.exp(-np.sum(A[i]*(x - P[i])**2, axis=1))
    return y

def kursawe_f1(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    s = np.sum(-10*np.exp(-0.2*np.sqrt(x[:,:-1]**2 + x[:,1:]**2)), axis=1)
    return s

BENCHMARKS: Dict[str, Benchmark] = {
    "rosenbrock2": Benchmark("rosenbrock2", 2, np.array([[-2.0, 2.0], [-1.0, 3.0]]), lambda X: rosenbrock(X)),
    "rosenbrock3": Benchmark("rosenbrock3", 3, np.array([[-2.0, 2.0], [-1.0, 3.0], [-2.0, 2.0]]), lambda X: rosenbrock(X)),
    "branin":      Benchmark("branin", 2, np.array([[-5.0, 10.0],[0.0, 15.0]]), branin),
    "hartmann3":   Benchmark("hartmann3", 3, np.array([[0.0, 1.0],[0.0, 1.0],[0.0, 1.0]]), hartmann3),
    "kursawe":     Benchmark("kursawe", 3, np.array([[-5.0, 5.0],[-5.0, 5.0],[-5.0, 5.0]]), kursawe_f1),
}


In [ ]:

import numpy as np

def sample_uniform(bounds: np.ndarray, n: int, rng=None) -> np.ndarray:
    rng = np.random.default_rng() if rng is None else rng
    low, high = bounds[:,0], bounds[:,1]
    u = rng.random((n, bounds.shape[0]))
    return low + u*(high - low)

def lhs(bounds: np.ndarray, n: int, rng=None) -> np.ndarray:
    rng = np.random.default_rng() if rng is None else rng
    d = bounds.shape[0]
    X = np.zeros((n, d))
    for j in range(d):
        perm = rng.permutation(n)
        points = (perm + rng.random(n)) / n
        low, high = bounds[j,0], bounds[j,1]
        X[:, j] = low + points*(high - low)
    return X

def generate_dataset(bench, n: int, sampler: str="lhs", rng=None):
    rng = np.random.default_rng() if rng is None else rng
    if sampler == "lhs":
        X = lhs(bench.bounds, n, rng)
    elif sampler == "random":
        X = sample_uniform(bench.bounds, n, rng)
    else:
        raise ValueError("sampler must be 'lhs' or 'random'")
    y = bench.f(X)
    return X, y

def add_outliers(X: np.ndarray, y: np.ndarray, frac_x: float=0.0, frac_y: float=0.05,
                 x_scale: float=3.0, y_scale: float=5.0, rng=None):
    rng = np.random.default_rng() if rng is None else rng
    n = X.shape[0]
    mask = np.zeros(n, dtype=bool)

    X_out = X.copy()
    y_out = y.copy()

    kx = int(np.round(frac_x * n))
    if kx > 0:
        idx_x = rng.choice(n, size=kx, replace=False)
        mask[idx_x] = True
        ranges = (X.max(axis=0) - X.min(axis=0))
        shift = (rng.standard_normal((kx, X.shape[1])) * (x_scale * (ranges + 1e-9)))
        X_out[idx_x] = X_out[idx_x] + shift

    ky = int(np.round(frac_y * n))
    if ky > 0:
        idx_y = rng.choice(n, size=ky, replace=False)
        mask[idx_y] = True
        sigma = np.std(y) + 1e-9
        y_out[idx_y] = y_out[idx_y] + rng.standard_normal(ky) * (y_scale * sigma)

    return X_out, y_out, mask


In [ ]:

import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope

def detect_iqr(y: np.ndarray, k: float=1.5) -> np.ndarray:
    q1, q3 = np.percentile(y, 25), np.percentile(y, 75)
    iqr = q3 - q1
    lower, upper = q1 - k*iqr, q3 + k*iqr
    return (y < lower) | (y > upper)

def detect_isoforest(X: np.ndarray, contamination: float=0.05, random_state: int=42):
    clf = IsolationForest(contamination=contamination, random_state=random_state)
    pred = clf.fit_predict(X)
    return pred == -1, clf

def detect_lof(X: np.ndarray, contamination: float=0.05, n_neighbors: int=20):
    lof = LocalOutlierFactor(contamination=contamination, n_neighbors=n_neighbors)
    pred = lof.fit_predict(X)
    return pred == -1, lof

def detect_ocsvm(X: np.ndarray, nu: float=0.05, gamma="scale"):
    oc = OneClassSVM(nu=nu, kernel="rbf", gamma=gamma)
    pred = oc.fit_predict(X)
    return pred == -1, oc

def detect_robust_cov(X: np.ndarray, contamination: float=0.05):
    ee = EllipticEnvelope(contamination=contamination, support_fraction=1.0, random_state=42)
    pred = ee.fit_predict(X)
    return pred == -1, ee


In [ ]:

import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel as C
from sklearn.metrics import r2_score

try:
    from scipy.stats import chi2
    SCIPY_AVAILABLE = True
except Exception:
    SCIPY_AVAILABLE = False

def fit_gp(X: np.ndarray, y: np.ndarray) -> GaussianProcessRegressor:
    kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=np.ones(X.shape[1]), length_scale_bounds=(1e-2, 1e3))              + WhiteKernel(noise_level=1e-3, noise_level_bounds=(1e-9, 1e-1))
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=3, normalize_y=True, random_state=42)
    gp.fit(X, y)
    return gp

def residual_diagnostics(gp: GaussianProcessRegressor, X: np.ndarray, y: np.ndarray):
    y_pred, y_std = gp.predict(X, return_std=True)
    resid = y - y_pred
    sigma_resid = np.std(resid)
    return y_pred, y_std, resid, sigma_resid

def mahalanobis_pvals(X: np.ndarray):
    mu = np.mean(X, axis=0)
    cov = np.cov(X, rowvar=False) + 1e-9*np.eye(X.shape[1])
    inv = np.linalg.inv(cov)
    dif = X - mu
    d2 = np.einsum('ni,ij,nj->n', dif, inv, dif)
    if SCIPY_AVAILABLE:
        p = 1.0 - chi2.cdf(d2, df=X.shape[1])
    else:
        p = np.full_like(d2, np.nan, dtype=float)
    return d2, p

def risk_before_simulation(X_train: np.ndarray, gp: GaussianProcessRegressor=None, X_cand: np.ndarray=None):
    X_eval = X_train if X_cand is None else X_cand
    mu = np.mean(X_train, axis=0)
    cov = np.cov(X_train, rowvar=False) + 1e-9*np.eye(X_train.shape[1])
    inv = np.linalg.inv(cov)
    dif = X_eval - mu
    d2_eval = np.einsum('ni,ij,nj->n', dif, inv, dif)

    risk = {"mahalanobis_d2": d2_eval}
    if SCIPY_AVAILABLE:
        risk["mahalanobis_p"] = 1.0 - chi2.cdf(d2_eval, df=X_train.shape[1])
    if gp is not None:
        _, std = gp.predict(X_eval, return_std=True)
        risk["gp_sigma"] = std
    return risk


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.gaussian_process import GaussianProcessRegressor

def plot_scatter_2d(X: np.ndarray, y: np.ndarray, mask_outliers=None, title="2D scatter (color=y)"):
    if X.shape[1] != 2:
        raise ValueError("plot_scatter_2d requires 2D inputs.")
    plt.figure(figsize=(6,5))
    sc = plt.scatter(X[:,0], X[:,1], s=30, c=y)
    plt.xlabel("x1"); plt.ylabel("x2"); plt.title(title)
    plt.colorbar(sc, label="y")
    if mask_outliers is not None:
        idx = np.where(mask_outliers)[0]
        plt.scatter(X[idx,0], X[idx,1], s=80, facecolors='none', edgecolors='k')
    plt.tight_layout(); plt.show()

def plot_residuals(y_true, y_pred, sigma_resid, k=2.0, title="Residuals vs Predicted (±kσ)"):
    resid = y_true - y_pred
    plt.figure(figsize=(6,5))
    plt.scatter(y_pred, resid, s=25)
    plt.axhline(0.0)
    plt.axhline(+k*sigma_resid, linestyle="--")
    plt.axhline(-k*sigma_resid, linestyle="--")
    plt.xlabel("Predicted"); plt.ylabel("Residual")
    plt.title(title)
    plt.tight_layout(); plt.show()

def plot_isoforest_scores(clf: IsolationForest):
    scores = clf.score_samples(clf._fit_X)
    plt.figure(figsize=(6,4))
    plt.hist(scores, bins=30)
    plt.xlabel("IsolationForest score"); plt.ylabel("Count")
    plt.title("Distribution of IsolationForest scores")
    plt.tight_layout(); plt.show()

def plot_gp_variance_2d(gp: GaussianProcessRegressor, bounds: np.ndarray, grid_n: int=80, title="GP predictive std"):
    if bounds.shape[0] != 2:
        raise ValueError("plot_gp_variance_2d requires 2D bounds.")
    x1 = np.linspace(bounds[0,0], bounds[0,1], grid_n)
    x2 = np.linspace(bounds[1,0], bounds[1,1], grid_n)
    X1, X2 = np.meshgrid(x1, x2)
    Xg = np.column_stack([X1.ravel(), X2.ravel()])
    _, std = gp.predict(Xg, return_std=True)
    Z = std.reshape(X1.shape)
    plt.figure(figsize=(6,5))
    cs = plt.contourf(X1, X2, Z, levels=20)
    plt.colorbar(cs, label="Pred. std (σ)")
    plt.xlabel("x1"); plt.ylabel("x2"); plt.title(title)
    plt.tight_layout(); plt.show()


## Demo: choose a benchmark, create DoE, inject outliers, run detectors, and visualize
You can change `BENCHMARK_NAME`, sampling size `N`, and outlier fractions.

In [ ]:

# === User parameters ===
BENCHMARK_NAME = "branin"     # "branin", "rosenbrock2", "rosenbrock3", "hartmann3", "kursawe"
N = 200
SAMPLER = "lhs"               # "lhs" or "random"
FRAC_X = 0.00                 # fraction of input-space outliers
FRAC_Y = 0.05                 # fraction of response-space outliers

bench = BENCHMARKS[BENCHMARK_NAME]
X, y = generate_dataset(bench, N, SAMPLER)
Xo, yo, mask_injected = add_outliers(X, y, frac_x=FRAC_X, frac_y=FRAC_Y)

print(f"Benchmark: {bench.name}  | dim={bench.dim}")
print(f"Bounds:\n{bench.bounds}")
print(f"Injected outliers: {mask_injected.sum()} / {N}")

# Visualize if 2D
if bench.dim == 2:
    plot_scatter_2d(Xo, yo, mask_outliers=mask_injected, title=f"{bench.name}: DoE with injected outliers")


In [ ]:

# --- Detectors ---
mask_iqr = detect_iqr(yo, k=1.5)
mask_iso, clf_iso = detect_isoforest(Xo, contamination=max(0.01, FRAC_Y + FRAC_X))
mask_lof, lof = detect_lof(Xo, contamination=max(0.01, FRAC_Y + FRAC_X))
mask_oc, oc = detect_ocsvm(Xo, nu=max(0.01, FRAC_Y + FRAC_X))
mask_rc, ee = detect_robust_cov(Xo, contamination=max(0.01, FRAC_Y + FRAC_X))

def report(name, mask):
    tp = np.sum(mask & mask_injected)
    fp = np.sum(mask & ~mask_injected)
    fn = np.sum(~mask & mask_injected)
    print(f"{name:18s}  flagged={mask.sum():3d} | TP={tp:3d}  FP={fp:3d}  FN={fn:3d}")

print("\nDetection summary (vs injected ground-truth mask):")
report("IQR(y)", mask_iqr)
report("IsolationForest", mask_iso)
report("LOF", mask_lof)
report("One-Class SVM", mask_oc)
report("Robust Covariance", mask_rc)

# Residual diagnostics via GP
gp = fit_gp(Xo[~mask_iso], yo[~mask_iso])  # fit on points not flagged by IF (simple robustification)
y_pred, y_std, resid, sigma_resid = residual_diagnostics(gp, Xo, yo)
from sklearn.metrics import r2_score
print(f"\nGP R^2 on all points: {r2_score(yo, y_pred):.3f}, residual σ={sigma_resid:.3g}")

# Plots
plot_residuals(yo, y_pred, sigma_resid, k=2.0, title="Residuals vs Predicted (±2σ)")
plot_isoforest_scores(clf_iso)

if bench.dim == 2:
    plot_gp_variance_2d(gp, bench.bounds, grid_n=80, title="GP predictive std (pre-simulation risk proxy)")


## Pre-simulation outlier risk for candidate designs
- **Mahalanobis distance** in input space (p-value if SciPy available)
- **GP predictive std** (if GP available) as uncertainty-driven risk

In [ ]:

# Build a small candidate set (e.g., random within bounds)
m = 10
X_cand = sample_uniform(bench.bounds, m)

risk = risk_before_simulation(Xo[~mask_iso], gp=gp, X_cand=X_cand)
print("Risk keys:", list(risk.keys()))
print("Top-5 risky by Mahalanobis d^2:")
top_idx = np.argsort(-risk["mahalanobis_d2"])[:5]
for i in top_idx:
    msg = f"idx={i:2d}  d2={risk['mahalanobis_d2'][i]:8.3f}"
    if 'mahalanobis_p' in risk:
        msg += f"  p={risk['mahalanobis_p'][i]:.3e}"
    if 'gp_sigma' in risk:
        msg += f"  gpσ={risk['gp_sigma'][i]:.3f}"
    print(msg)

# Optional plot if 2D: overlay GP std as contours with candidate points (separate single plot)
if bench.dim == 2:
    plot_gp_variance_2d(gp, bench.bounds, grid_n=80, title="GP σ (pre-sim risk)")
    plt.figure(figsize=(6,5))
    plt.scatter(X_cand[:,0], X_cand[:,1], s=60)
    plt.xlabel("x1"); plt.ylabel("x2"); plt.title("Candidate designs (for risk eval)")
    plt.tight_layout(); plt.show()
